In [11]:
import sys
from pathlib import Path

# Ensure PyVRP is used from the installed environment, not the local clone.
# ROOT = Path().resolve()
# sys.path.insert(0, str(ROOT / "libs" / "pyvrp"))

import matplotlib.pyplot as plt
from tabulate import tabulate
from vrplib import read_solution
from pyvrp import read, Solution, GeneticAlgorithm, RandomNumberGenerator, PenaltyManager, Population, PopulationParams, SolveParams
from pyvrp.solve import solve as pyvrp_solve
import pyvrp.diversity as diversity
from pyvrp.plotting import (
    plot_coordinates,
    plot_instance,
    plot_result,
    plot_route_schedule,
)
from pyvrp.stop import MaxRuntime
from pyvrp.search import LocalSearch

from pathlib import Path
import time
import csv
import os

In [ ]:
from utils import run_pyvrp

In [ ]:
# Experiment settings - change if needed
instance_path = 'instances\E-n13-k4.vrp'
seed = 0
TIME_LIMIT = 30*60  # seconds per run (MaxRuntime)
time_limit = 1
results_dir = Path('results')
os.makedirs(results_dir, exist_ok=True)
results_csv = results_dir / 'experiments.csv'

def run_config(name, initial_sol, intensify, diversify, seed, time_limit=TIME_LIMIT):
    stop = MaxRuntime(time_limit)
    t0 = time.time()
    res = run_pyvrp(instance_path, stop, seed=seed, initial_solution=initial_sol, intensify=intensify, diversify=diversify)
    runtime = time.time() - t0
    return {
        'config': name,
        'cost': res.cost(),
        'feasible': res.is_feasible(),
        'runtime_s': runtime,
        'seed': seed
    }

In [ ]:
# Define the 4 configurations and run them
configs = [
    ('both', None, True, True),
    ('intensify_only', None, True, False),
    ('diversify_only', None, False, True),
    ('none', None, False, False),
]

rows = []
for name, const_heuristic, intensify, diversify in configs:
    print(f'Running {name} (intensify={intensify}, diversify={diversify})...')
    try:
        r = run_config(name, const_heuristic, intensify, diversify, seed, time_limit)
        rows.append(r)
        print('  done:', r)
    except Exception as e:
        print(f'  ERROR running {name}:', e)
        rows.append({'config': name, 'cost': None, 'feasible': False, 'runtime_s': None, 'seed': seed, 'error': str(e)})

# Write CSV
fieldnames = ['config', 'cost', 'feasible', 'runtime_s', 'seed']
with open(results_csv, 'w', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for r in rows:
        writer.writerow({k: r.get(k) for k in fieldnames})

print('Results written to', results_csv)

Running both (intensify=True, diversify=True)...
  done: {'config': 'both', 'cost': 290, 'feasible': True, 'runtime_s': 1.004150629043579, 'seed': 0}
Running intensify_only (intensify=True, diversify=False)...
  done: {'config': 'both', 'cost': 290, 'feasible': True, 'runtime_s': 1.004150629043579, 'seed': 0}
Running intensify_only (intensify=True, diversify=False)...
  done: {'config': 'intensify_only', 'cost': 290, 'feasible': True, 'runtime_s': 1.0037927627563477, 'seed': 0}
Running diversify_only (intensify=False, diversify=True)...
  done: {'config': 'intensify_only', 'cost': 290, 'feasible': True, 'runtime_s': 1.0037927627563477, 'seed': 0}
Running diversify_only (intensify=False, diversify=True)...
  done: {'config': 'diversify_only', 'cost': 290, 'feasible': True, 'runtime_s': 1.0032298564910889, 'seed': 0}
Running none (intensify=False, diversify=False)...
  done: {'config': 'diversify_only', 'cost': 290, 'feasible': True, 'runtime_s': 1.0032298564910889, 'seed': 0}
Running no

In [16]:
# Display the CSV we just wrote
import pandas as pd
df = pd.read_csv(results_csv)
df

,config,cost,feasible,runtime_s,seed
0,both,290,True,1.004151,0
1,intensify_only,290,True,1.003793,0
2,diversify_only,290,True,1.003230,0
3,none,290,True,1.005006,0
